In [3]:
import numpy as np
import pandas as pd
import os
import csv
from datetime import timedelta, datetime, date
import datetime as dt
import pickle

In [4]:
#Import ESC File#
esc_file = pd.read_csv("ESC 08072020.csv",dtype={'Pernr SAP': str})
esc_file = esc_file[['Pernr SAP', 'Covid Y/N','Ticket Created - Date']]

#Import roster file
roster_file = pd.read_csv("Roster_08072020_workbook.csv",dtype={'Employee PERNR - Key': str,'WL County Fips':str, "HL County Fips":str })
roster_file = roster_file[['Employee PERNR - Key','Annual ABBR',
                          'Annual Salary','Date of Birth (*)', 'Division (As of Today)','Division-1 (As of Today)',
                          'Employment Status','Headcount',
                          'Job Code','Job Family','Job Function','Job Group 1','Job Group 2','Job Level','Original Hire Date','Virtual Employee Type',
                          'WL Zip Truncated','WL County Fips','WL County Name','HL Zip Truncated','HL County Fips','HL County Name']]

# roster_file = roster_file[['Employee PERNR - Key','WL Zip Truncated','WL County Fips','WL County Name','HL Zip Truncated',
#                            'HL County Fips','HL County Name','Headcount','Job Function','Job Group 1']]
#Import base file
base = pd.read_csv("Prevalence_base.csv", dtype={'fips':str,'Population':int})


# print(esc_file.head())
# print(roster_file.head())
# print(base.head())

    fips   County    State state_code  Population  Comcast Pop (HL)  \
0  01001  Autauga  Alabama         AL       55869                 0   
1  01003  Baldwin  Alabama         AL      223234                10   
2  01005  Barbour  Alabama         AL       24686                 0   
3  01007     Bibb  Alabama         AL       22394                 4   
4  01009   Blount  Alabama         AL       57826                 2   

   Comcast Pop (WL) CCC < 50 Employees  
0                 0                Yes  
1                 0                Yes  
2                 0                Yes  
3                 0                Yes  
4                 0                Yes  


C:\Users\dherlu494\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25,29,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#Change name of Pernr columns in both files

roster_file1 = roster_file.rename({'Employee PERNR - Key':'PERNR','Date of Birth (*)':'DOB','Division (As of Today)':'Division',
                                   'Division-1 (As of Today)':'Division 1','Original Hire Date':'DOH'}, axis = 1)
esc_file1 = esc_file.rename({'Pernr SAP':'PERNR','Ticket Created - Date':'date'},axis = 1)

In [6]:
# print(esc_file1.head())
# print(roster_file1.head())

      PERNR  Covid Y/N       date
0  10001471          0   5/1/2020
1  10001709          0   5/1/2020
2  10001712          0  5/13/2020
3  10001715          0  5/21/2020
4  10001728          0   5/1/2020


In [7]:
esc_file1['date'] = pd.to_datetime(esc_file1['date'], format = '%m/%d/%Y')

In [8]:
merge1 = roster_file1.merge(esc_file1, 
                          how = 'left', on = 'PERNR')
merge1['date'] = pd.to_datetime(merge1['date'], format = '%m/%d/%Y')
# print(merge1.head())

      PERNR  Annual ABBR  Annual Salary              DOB            Division  \
0  10000054    186447.31       20000.00   3/13/1960 0:00         Advertising   
1  10000063         0.00       57532.80    8/2/1953 0:00        Headquarters   
2  10000401         0.00      124355.35  10/30/1978 0:00  Northeast Division   
3  10000480         0.00       87124.96  12/28/1979 0:00        Headquarters   
4  10000720         0.00      103523.16   2/28/1985 0:00       West Division   

                     Division 1 Employment Status  Headcount  \
0                       Markets            Active          1   
1        Finance and Accounting            Active          1   
2     Northeast Division Office            Active          1   
3  Technology Product Xperience            Active          1   
4           West Division Other            Active          1   

                                Job Code               Job Family  ...  \
0        Acct Exec 2, Ad Sales- Regional       Ad Sales- Reg

In [9]:
#fill in the Date NaT with 8/7/2020 aka most recent date#
dates = '08/07/2020'
merge1['date'] = merge1['date'].fillna(pd.to_datetime(dates,format = '%m/%d/%Y'))
# print(merge1.head())

      PERNR  Annual ABBR  Annual Salary              DOB            Division  \
0  10000054    186447.31       20000.00   3/13/1960 0:00         Advertising   
1  10000063         0.00       57532.80    8/2/1953 0:00        Headquarters   
2  10000401         0.00      124355.35  10/30/1978 0:00  Northeast Division   
3  10000480         0.00       87124.96  12/28/1979 0:00        Headquarters   
4  10000720         0.00      103523.16   2/28/1985 0:00       West Division   

                     Division 1 Employment Status  Headcount  \
0                       Markets            Active          1   
1        Finance and Accounting            Active          1   
2     Northeast Division Office            Active          1   
3  Technology Product Xperience            Active          1   
4           West Division Other            Active          1   

                                Job Code               Job Family  ...  \
0        Acct Exec 2, Ad Sales- Regional       Ad Sales- Reg

In [10]:
#aggregate at fips level#
merge = merge1
WL = merge[['WL County Fips','Covid Y/N','Headcount']]
WL_df = WL.groupby(['WL County Fips'], as_index=False).sum()
WL_df = WL_df.rename({'Covid Y/N':'WL_Covid','Headcount':'WL_Count','WL County Fips': 'fips'}, axis = 1)

HL = merge[['HL County Fips','Covid Y/N','Headcount']]
HL_df = HL.groupby(['HL County Fips'], as_index=False).sum()
HL_df = HL_df.rename({'Covid Y/N':'HL_Covid','Headcount':'HL_Count','HL County Fips': 'fips'}, axis = 1)

# print(WL_df.head())
# print(HL_df.head())


    fips  WL_Covid  WL_Count
0  01019       0.0        33
1  01041       0.0         1
2  01061       0.0        23
3  01077       0.0        21
4  01089       1.0       159
    fips  HL_Covid  HL_Count
0  01003       0.0        10
1  01007       0.0         4
2  01009       0.0         2
3  01015       0.0         6
4  01017       0.0         2


In [11]:
#JOB GROUP 1 #
WL_Tech = merge[merge["Job Group 1"]== "Techs"]
WL_Techs = WL_Tech[['WL County Fips','Covid Y/N','Headcount']]
WL_df_Tech = WL_Techs.groupby(['WL County Fips'], as_index=False).sum()
WL_df_Techs = WL_df_Tech.rename({'Covid Y/N':'WL_Techs_Covid','WL County Fips': 'fips','Headcount':'Tech_Count'}, axis = 1)
print(WL_df_Techs.head())


WL_CAE = merge[merge["Job Group 1"]== "CAE"]
WL_CAES = WL_CAE[['WL County Fips','Covid Y/N','Headcount']]
WL_df_CAE = WL_CAES.groupby(['WL County Fips'], as_index=False).sum()
WL_df_CAES = WL_df_CAE.rename({'Covid Y/N':'WL_CAE_Covid','WL County Fips': 'fips','Headcount':'CAE_Count'}, axis = 1)
print(WL_df_CAES.head())


WL_CableStore = merge[merge["Job Group 1"]== "Cable Stores"]
WL_CableStores = WL_CableStore[['WL County Fips','Covid Y/N','Headcount']]
WL_df_CableS = WL_CableStores.groupby(['WL County Fips'], as_index=False).sum()
WL_df_CableStores = WL_df_CableS.rename({'Covid Y/N':'WL_CABLEStores_Covid','WL County Fips': 'fips','Headcount':'Cable_Count'}, axis = 1)
print(WL_df_CableStores.head())

# WL_BusServ = merge[merge["Job Group 1"]== "Business Services"]
# WL_BusServs = WL_BusServ[['WL County Fips','Covid Y/N']]
# WL_df_BusServ = WL_BusServs.groupby(['WL County Fips'], as_index=False).sum()
# WL_df_BusServs = WL_df_BusServ.rename({'Covid Y/N':'WL_BusServ_Covid','WL County Fips': 'fips'}, axis = 1)
# print(WL_df_BusServs.head())


    fips  WL_Techs_Covid  Tech_Count
0  01019             0.0          20
1  01061             0.0          17
2  01077             0.0          15
3  01089             0.0          41
4  01097             0.0          59
    fips  WL_CAE_Covid  CAE_Count
0  01089           1.0         67
1  01097           0.0          2
2  01125           1.0          3
3  04019           2.0        222
4  05035           1.0          2
    fips  WL_CABLEStores_Covid  Cable_Count
0  01019                   0.0            4
1  01061                   0.0            5
2  01089                   0.0            6
3  01097                   0.0           12
4  01125                   0.0            8


In [12]:
#filtered for only what we care for and july
#Add Code here for joining to FIPS base file to map back if needed#

df_CCC_cases1 = pd.merge(base, WL_df[['fips','WL_Covid','WL_Count']], on = 'fips', how = 'left')

df_CCC_cases2 = pd.merge(df_CCC_cases1, HL_df[['fips','HL_Covid','HL_Count']], on = 'fips', how = 'left')

df_CCC_cases3 = pd.merge(df_CCC_cases2, WL_df_CAES[['fips','WL_CAE_Covid','CAE_Count']], on = 'fips', how = 'left')

df_CCC_cases4 = pd.merge(df_CCC_cases3, WL_df_Techs[['fips','WL_Techs_Covid','Tech_Count']], on = 'fips', how = 'left')

df_CCC_cases5 = pd.merge(df_CCC_cases4, WL_df_CableStores[['fips','WL_CABLEStores_Covid','Cable_Count']], on = 'fips', how = 'left')

df_CCC_cases = pd.merge(merge, df_CCC_cases5, left_on = 'WL County Fips',right_on = 'fips', how = 'left')


In [14]:
#Pull in NYTimes Data#
#Bring in County Data set
nytimes = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
counties = pd.read_csv(nytimes,dtype={'fips': str})

counties.loc[counties['county'] == 'New York City', 'fips'] = '36061'

In [15]:
#Filter for specific date for prevalence data pull#
counties.sort_values(by=['fips','date'],inplace=True, ascending=True)


# counties_filtered = counties[counties['date'].isin(['2020-08-07'])]
counties['date'] = pd.to_datetime(counties['date'], format = '%Y-%m-%d')
# counties['date'] =counties['date'].dt.strftime('%m/%d/%Y')

print(counties.head())

            date   county    state   fips  cases  deaths
9480  2020-03-24  Autauga  Alabama  01001      1       0
10835 2020-03-25  Autauga  Alabama  01001      4       0
12367 2020-03-26  Autauga  Alabama  01001      6       0
14025 2020-03-27  Autauga  Alabama  01001      6       0
15803 2020-03-28  Autauga  Alabama  01001      6       0


In [17]:
#Merge with the esc_roster dataset

df = pd.merge(df_CCC_cases, counties[['date','fips','cases','deaths']],left_on = ['fips','date'],right_on = ['fips','date'], how = 'left' )
# print(df.head())


      PERNR  Annual ABBR  Annual Salary              DOB            Division  \
0  10000054    186447.31       20000.00   3/13/1960 0:00         Advertising   
1  10000063         0.00       57532.80    8/2/1953 0:00        Headquarters   
2  10000401         0.00      124355.35  10/30/1978 0:00  Northeast Division   
3  10000480         0.00       87124.96  12/28/1979 0:00        Headquarters   
4  10000720         0.00      103523.16   2/28/1985 0:00       West Division   

                     Division 1 Employment Status  Headcount  \
0                       Markets            Active          1   
1        Finance and Accounting            Active          1   
2     Northeast Division Office            Active          1   
3  Technology Product Xperience            Active          1   
4           West Division Other            Active          1   

                                Job Code               Job Family  ...  \
0        Acct Exec 2, Ad Sales- Regional       Ad Sales- Reg

In [18]:

#Prevalence numbers- County
df['County_Prevalence'] = df['cases']/df['Population']
df['Death/Inf'] = df['deaths']/df['cases']
df['Deaths_Prev'] =df['deaths']/df['Population']

# #need to divide by state pop#
# df['Hosp/Inf'] = df['hospitalizedCumulative']/df['cases']

#Prevalence numbers- Comcast County
df['WL_Prev'] = df['WL_Covid']/df['Comcast Pop (WL)']
df['HL_Prev'] = df['HL_Covid']/df['Comcast Pop (HL)']

#Prevalence by Job Function
# df['FieldOps_Prev'] = df['WL_FO_Covid']/df['Field Ops']
# df['CusService_Prev'] = df['WL_CS_Covid']/df['Customer Service']
# df['ProductSales_Prev'] = df['WL_PS_Covid']/df['Product Sales']
# df['Engineer_Prev'] = df['WL_ENG_Covid']/df['Engineering']
# df['FinanceAcct_Prev'] = df['WL_FA_Covid']/df['Finance & Accounting']

df['Techs_Prev'] = df['WL_Techs_Covid']/df['Tech_Count']
df['CAE_Prev'] = df['WL_CAE_Covid']/df['CAE_Count']
df['CABLESTORES_Prev'] = df['WL_CABLEStores_Covid']/df['Cable_Count']
# df['BUSSERV_Prev'] = df['WL_BusServ_Covid']/df['Business Services']

In [19]:
## CASES P 100K
df['Cases p 100 K_County'] = df['County_Prevalence']*100000
df['WL_Cases_p_100k'] = df['WL_Prev']*100000

In [22]:
#Change Nulls to 0#

df_2 = df.replace([np.inf, -np.inf], np.nan)

df_testing = df_2.fillna(0)


In [23]:
date = '08072020'
# df_final = df_testing3[df_testing3['Comcast Pop (WL)'] >= 50]
df_final = df_testing
# df_final.to_csv("Prevalence_"+date+"Modeling_dataset.csv")

In [25]:
# #Create Pickle to Run Linear Regression#
pickle.dump(df_final, open("comcast_regression.sav", "wb"))

# Code for New Cases p 100K on Monthly Basis#

In [26]:
# #Group by to get new cases 
# #take diff of cases
# counties_April['April_diffs'] = counties_April.cases.diff()
# counties_May['May_diffs'] = counties_May.cases.diff()
# counties_June['June_diffs'] = counties_June.cases.diff()
# counties_July['July_diffs'] = counties_July.cases.diff()
counties['diffs'] = counties.cases.diff()

#counties_pop = pd.merge(base['fips','County','State','state_code','Population'], counties['fips','diffs'], on = 'fips', how = 'right')
# counties.to_csv("Counties_Newcases7.csv")
# print(counties.head())

            date   county    state   fips  cases  deaths  diffs
9480  2020-03-24  Autauga  Alabama  01001      1       0    NaN
10835 2020-03-25  Autauga  Alabama  01001      4       0    3.0
12367 2020-03-26  Autauga  Alabama  01001      6       0    2.0
14025 2020-03-27  Autauga  Alabama  01001      6       0    0.0
15803 2020-03-28  Autauga  Alabama  01001      6       0    0.0


In [27]:
# #New cases in last month--From NYT Data
counties_April = counties[(counties['date'] >= '2020-04-01') & (counties['date']< '2020-05-01')]
counties_May = counties[(counties['date'] >= '2020-05-01') & (counties['date']< '2020-06-01')]
counties_June = counties[(counties['date'] >= '2020-06-01') & (counties['date']< '2020-07-01')]
counties_July = counties[(counties['date'] >= '2020-07-01') & (counties['date']< '2020-08-01')]


In [28]:
counties_April_group = counties_April.groupby(['fips'], as_index=False).mean()
counties_May_group = counties_May.groupby(['fips'], as_index=False).mean()
counties_June_group = counties_June.groupby(['fips'], as_index=False).mean()
counties_July_group = counties_July.groupby(['fips'], as_index=False).mean()

In [29]:
counties_April_group['April_diffs'] = counties_April_group['diffs']
counties_May_group['May_diffs'] = counties_May_group['diffs']
counties_June_group['June_diffs'] = counties_June_group['diffs']
counties_July_group['July_diffs'] = counties_July_group['diffs']

In [30]:
# df_final.to_csv("checking1.csv")
# print(df_final.dtypes)
# print(counties_April_group.dtypes)

In [31]:
df_prev_hist1 = pd.merge(df_final, counties_April_group[['fips','April_diffs']], on = 'fips', how = 'left')
df_prev_hist2 = pd.merge(df_prev_hist1, counties_May_group[['fips','May_diffs']], on = 'fips', how = 'left')
df_prev_hist3 = pd.merge(df_prev_hist2, counties_June_group[['fips','June_diffs']], on = 'fips', how = 'left')
df_prev_hist = pd.merge(df_prev_hist3, counties_July_group[['fips','July_diffs']], on = 'fips', how = 'left')

print(df_prev_hist.head())

      PERNR  Annual ABBR  Annual Salary              DOB            Division  \
0  10000054    186447.31       20000.00   3/13/1960 0:00         Advertising   
1  10000063         0.00       57532.80    8/2/1953 0:00        Headquarters   
2  10000401         0.00      124355.35  10/30/1978 0:00  Northeast Division   
3  10000480         0.00       87124.96  12/28/1979 0:00        Headquarters   
4  10000720         0.00      103523.16   2/28/1985 0:00       West Division   

                     Division 1 Employment Status  Headcount  \
0                       Markets            Active          1   
1        Finance and Accounting            Active          1   
2     Northeast Division Office            Active          1   
3  Technology Product Xperience            Active          1   
4           West Division Other            Active          1   

                                Job Code               Job Family  ...  \
0        Acct Exec 2, Ad Sales- Regional       Ad Sales- Reg

In [32]:
# #Monthly New Cases per 100K ##
df["April_New_Cases_100k"] =  (df_prev_hist["April_diffs"]/df_prev_hist["Population"]) *100000
df["May_New_Cases_100k"] = (df_prev_hist["May_diffs"]/df_prev_hist["Population"]) *100000
df["June_New_Cases_100k"] = (df_prev_hist["June_diffs"]/df_prev_hist["Population"]) *100000
df["July_New_Cases_100k"] = (df_prev_hist["July_diffs"]/df_prev_hist["Population"]) *100000


In [41]:
counties['diffs'] = counties['diffs'].fillna(0)
counties.reset_index(drop=True)

,date,county,state,fips,cases,deaths,diffs,expanding_mean,diff
0,2020-03-24,Autauga,Alabama,01001,1,0,0.0,NaN,0.0
1,2020-03-25,Autauga,Alabama,01001,4,0,3.0,NaN,3.0
2,2020-03-26,Autauga,Alabama,01001,6,0,2.0,NaN,2.0
3,2020-03-27,Autauga,Alabama,01001,6,0,0.0,NaN,0.0
4,2020-03-28,Autauga,Alabama,01001,6,0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
427835,2020-08-12,Unknown,Tennessee,NaN,88,0,-1505.0,NaN,-1505.0
427836,2020-08-12,Unknown,Utah,NaN,16,0,-72.0,NaN,-72.0
427837,2020-08-12,Unknown,Vermont,NaN,8,0,-8.0,NaN,-8.0
427838,2020-08-12,Unknown,Washington,NaN,2,2,-6.0,NaN,-6.0


In [42]:
##Rolling Averages
# df_new_cases = pd.DataFrame(counties.iloc[:,-1])
# df_new_cases.dropna(inplace=True)
# print(df_new_cases.head())
counties['expanding_mean'] = counties.groupby('fips')['diffs'].rolling(window = 7, min_periods = 7).mean().reset_index(0,drop=True)

In [43]:
print(counties.head())

            date   county    state   fips  cases  deaths  diffs  \
9480  2020-03-24  Autauga  Alabama  01001      1       0    0.0   
10835 2020-03-25  Autauga  Alabama  01001      4       0    3.0   
12367 2020-03-26  Autauga  Alabama  01001      6       0    2.0   
14025 2020-03-27  Autauga  Alabama  01001      6       0    0.0   
15803 2020-03-28  Autauga  Alabama  01001      6       0    0.0   

       expanding_mean  diff  
9480              NaN   0.0  
10835             NaN   3.0  
12367             NaN   2.0  
14025             NaN   0.0  
15803             NaN   0.0  


In [44]:
#Join onto Master Dataset the Rolling average based on date and fips of person who had Covid
df_finalized = pd.merge(df, counties[['fips','date','expanding_mean']], on = ['fips','date'], how = 'left')

##Calc New Cases p 100K
df_finalized['New_Cases_p_100K'] = (df_finalized['expanding_mean'] / df_finalized['Population']) *100000

In [45]:
df_finalized.to_csv("Iamthebombdotcom.csv")